# Tweet scraping

In [ ]:
import snscrape.modules.twitter as sntwitter
import pandas as pd
from ttp import ttp
from langdetect import detect, detect_langs
import re

In [ ]:
pd.set_option('display.max_colwidth', None)

### Get tweets by keyword

In [ ]:
def give_me_tweets_keyword(max_num, word_list):
    text=' OR '.join(word_list)
    tweets_list_ca=[]
    tweets_list_es=[]
    tweets_list_en=[]
    
    for i,tweet in enumerate(sntwitter.TwitterSearchScraper(text+' AND '+"-FC", "near:BCN within:15mi").get_items()):
        if i>=max_num:
            break
        tweet_text=quick_clean(tweet.content)
        #print(i, tweet_text)
        try:
            if len(tweet_text)>10:
                if 'es:' in str(detect_langs(tweet_text)):
                    tweets_list_es.append([tweet.date, tweet.content])
                if ('ca:' in str(detect_langs(tweet_text))):
                    tweets_list_ca.append([tweet.date, tweet.content])
                if ('en:' in str(detect_langs(tweet_text))):
                    tweets_list_en.append([tweet.date, tweet.content])
        except:
            print('No language features:', tweet_text)

    tweets_df_ca = pd.DataFrame(tweets_list_ca, columns=['Datetime',  'Text'])
    tweets_df_es = pd.DataFrame(tweets_list_es, columns=['Datetime',  'Text'])
    tweets_df_en = pd.DataFrame(tweets_list_en, columns=['Datetime',  'Text'])
    print(tweets_df_ca)
    print(tweets_df_es)
    print(tweets_df_en)
    
    
    return tweets_df_ca, tweets_df_es, tweets_df_en

In [ ]:
give_me_tweets_keyword(100, ['Barcelona'])

In [ ]:
def give_me_tweets_keyword(max_num, word_list):
    text=' OR '.join(word_list)
    tweets_list=[]
    for i,tweet in enumerate(sntwitter.TwitterSearchScraper(text).get_items()):
        if i>=max_num:
            break
        tweets_list.append([tweet.date, tweet.content])#tweet.id, tweet.user.username

    tweets_df = pd.DataFrame(tweets_list, columns=['Datetime',  'Text'])
    return tweets_df

### Trying to detect if a tweet included words in both Catalan and Spanish

In [ ]:
result=detect_langs()
result

In [ ]:
def is_mixed(text):
    if ('es:' in str(detect_langs(text))) and ('ca:' in str(detect_langs(text))):
        return True
    else:
        return False 

In [ ]:
#use Langdetect to filter tweets found near Barcelona

tweets_list=[]
for i,tweet in enumerate(sntwitter.TwitterSearchScraper("near:BCN within:15mi").get_items()):
    if i>=100:
        break
    if len(tweet.content)>0 and is_mixed(tweet.content)==True:
        tweets_list.append([tweet.date, tweet.content])#tweet.id, tweet.user.username

tweets_df = pd.DataFrame(tweets_list, columns=['Datetime', 'Text'])

In [ ]:
pd.set_option('display.max_colwidth', None)
print(tweets_df['Text'])

## Cleaning tweets

In [ ]:
p = ttp.Parser()
result = p.parse("@burnettedmond, you now support #IvoWertzel's tweet parser! https://github.com/edmondburnett/")
#print(result.reply)
#print(result.users)
#print(result.tags)
#print(result.urls)

tweet="@burnettedmond, you now support #IvoWertzel's tweet parser! https://github.com/edmondburnett/"
clean_tweet=tweet.replace(result.reply, '')
for item in result.users:
    clean_tweet=tweet.replace('@'+item, '')
#for item in result.tags:
#    clean_tweet=clean_tweet.replace(item, '')
for item in result.urls:
    clean_tweet=clean_tweet.replace(item, '')
    
clean_tweet

#### This function removes emojis

In [ ]:
def remove_emoji(text):
    emoji_pattern = re.compile("["
   u"\U0001F600-\U0001F64F"  # emoticons
                u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                u"\U0001F680-\U0001F6FF"  # transport & map symbols
                u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                u"\U00002702-\U000027B0"
                u"\U000024C2-\U0001F251"
                u"\U0001f926-\U0001f937"
                u'\U00010000-\U0010ffff'
                u"\u200d"
                u"\u2640-\u2642"
                u"\u2600-\u2B55"
                u"\u23cf"
                u"\u23e9"
                u"\u231a"
                u"\u3030"
                u"\ufe0f"
                        "]+", flags=re.UNICODE)
    return emoji_pattern.sub(r'', text)

### Final cleaning function

In [ ]:
def quick_clean(tweet_text):
    p = ttp.Parser()
    result = p.parse(tweet_text)
    for item in result.users:
        tweet_text=tweet_text.replace('@'+item, '')
    for item in result.urls:
        tweet_text=tweet_text.replace(item, '')
    tweet_text=remove_emoji(tweet_text)
    return tweet_text

### Clean the tweets and then select those that are mixed (have CA and ES in it) and that have been published near Barcelona

In [ ]:
tweets_list=[]
for i,tweet in enumerate(sntwitter.TwitterSearchScraper("near:BCN within:15mi").get_items()):
    if i>=1000:
        break
    tweet_text=quick_clean(tweet.content)
    try:
        if len(tweet_text)>10 and is_mixed(tweet_text)==True:
            tweets_list.append([tweet.date, tweet_text])#tweet.id, tweet.user.username
    except:
        print('No language features:', tweet_text)

tweets_df = pd.DataFrame(tweets_list, columns=['Datetime', 'Text'])

### Search by username (and also includes the mixed language thing)

In [ ]:
tweets_list=[]
for i,tweet in enumerate(sntwitter.TwitterSearchScraper("from:USER_NAME").get_items()):
    if i>=1000:
        break
    tweet_text=quick_clean(tweet.content)
    try:
        if len(tweet_text)>10 and is_mixed(tweet_text)==True:
            tweets_list.append([tweet.date, tweet_text])#tweet.id, tweet.user.username
    except:
        print('No language features:', tweet_text)

tweets_df = pd.DataFrame(tweets_list, columns=['Datetime', 'Text'])